In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_parquet('../data/SO_2014_2022.pq')

df = df[(df['Salary'] > 0) & (df['Salary'] < 250000)]
df['Year'] = df['Year'].astype(int)

df.head()

,Year,Salary,JobSat,YearsCode,YearsCodePro,Age,Education,OrgSize,LastNewJob,Employment,RespondentType,JobSeek,Gender,Student,Country,CodingActivities,DevType,LearnCodeFrom,LangPresent
0,2022,69318.0,<NA>,10,5.0,25-34,master,500 to 999 employees,None,fulltime,dev,None,male,no,Germany,School or academic work,Data scientist or machine learning specialist;...,"Books / Physical media;School (i.e., Universit...",C;C++;Java;JavaScript;MATLAB;Python;Scala;SQL;...
6,2022,27652.0,<NA>,18,10.0,25-34,bachelor,"1,000 to 4,999 employees",None,fulltime,dev,None,male,no,Colombia,Hobby,"Developer, full-stack;Developer, back-end",Books / Physical media;Other online resources ...,Bash/Shell/PowerShell;Elixir;HTML/CSS;JavaScri...
9,2022,15431.0,<NA>,5,5.0,25-34,bachelor,20 to 99 employees,None,fulltime,dev,None,male,no,Ghana,Freelance/contract work,"Developer, back-end",On the job training;Coding Bootcamp,JavaScript;Ruby
13,2022,47352.0,<NA>,7,7.0,45-54,master,10 to 19 employees,None,fulltime,non-dev,None,male,no,Belgium,Hobby,"Developer, back-end;Educator or academic;Datab...",Books / Physical media;On the job training;Col...,Delphi;SQL
22,2022,78084.0,<NA>,25,25.0,45-54,bachelor,500 to 999 employees,None,fulltime,non-dev,None,male,no,Canada,Hobby;Contribute to open-source projects,"Engineer, site reliability;Security professional",Books / Physical media;Other online resources ...,Bash/Shell/PowerShell;C;JavaScript;Perl;PHP;Py...


In [39]:
# Show the top-n countries in the plot.
TOP_N_COUNTRIES = 5

# Take the top-n countries that have the most developers.
countries = df[['Country', 'RespondentType']].query('RespondentType == "dev"') \
    .groupby('Country', as_index=False) \
    .count() \
    .sort_values('RespondentType', ascending=False) \
    .head(TOP_N_COUNTRIES) \
    ['Country'] \
    .tolist()

countries_df = df[df['Country'].isin(countries)]
    
# Calculate the salary gap in percentage.
salary_men_df = countries_df.query('Gender == "male"') \
    .groupby(['Year', 'Country']) \
    .agg({'Salary': 'mean' }) \
    .reset_index() \
    .rename(columns={'Salary': 'SalaryMen'})

salary_women_df = countries_df.query('Gender == "female"') \
    .groupby(['Year', 'Country']) \
    .agg({'Salary': 'mean' }) \
    .reset_index() \
    .rename(columns={'Salary': 'SalaryWomen'})

salary_gap_df = pd.merge(salary_men_df, salary_women_df, on=['Year', 'Country']).dropna()
salary_gap_df['SalaryGap'] = abs(salary_gap_df['SalaryMen'] - salary_gap_df['SalaryWomen'])
salary_gap_df['SalaryGapPercent'] = (salary_gap_df['SalaryMen'] - salary_gap_df['SalaryWomen']) / salary_gap_df['SalaryMen'] * 100

salary_gap_df.head()

,Year,Country,SalaryMen,SalaryWomen,SalaryGap,SalaryGapPercent
0,2014,Canada,71124.260355,57692.307692,13431.952663,18.885191
1,2014,Germany,61491.596639,50714.285714,10777.310924,17.526478
2,2014,India,21551.724138,20833.333333,718.390805,3.333333
3,2014,United Kingdom,71336.842105,46071.428571,25265.413534,35.417062
4,2014,United States,86482.707873,79157.303371,7325.404503,8.470369


In [55]:
# Get the median salary for men and women per country.
median_salary_men_df = countries_df.query('Gender == "male"') \
    .groupby(['Year', 'Country']) \
    .agg({'Salary': 'median' }) \
    .reset_index()

median_salary_men_df = countries_df.query('Gender == "male"') \
    .groupby(['Year', 'Country']) \
    .agg({'Salary': 'median' }) \
    .reset_index()

,Year,Country,Salary
0,2014,Canada,70000.000000
1,2014,Germany,50000.000000
2,2014,India,15000.000000
3,2014,United Kingdom,70000.000000
4,2014,United States,90000.000000
5,2015,Canada,70000.000000
6,2015,Germany,50000.000000
7,2015,India,15000.000000
8,2015,United Kingdom,70000.000000
9,2015,United States,90000.000000


In [53]:
fig = px.line(
    salary_gap_df,
    x='Year',
    y='SalaryGapPercent',
    color='Country',
    hover_data={'SalaryGapPercent': ':d'},
    title='Salary gap between men and women from 2014-2022'
)

fig.update_layout(yaxis_title='Salary gap in percentage')

fig.show()